# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 22 2022 8:30AM,244604,10,0085942552,ISDIN Corporation,Released
1,Aug 22 2022 8:30AM,244604,10,0085942549,ISDIN Corporation,Released
2,Aug 22 2022 8:30AM,244604,10,0085942587,ISDIN Corporation,Released
3,Aug 22 2022 8:30AM,244604,10,0085942600,ISDIN Corporation,Released
4,Aug 22 2022 8:30AM,244604,10,0085942618,ISDIN Corporation,Released
5,Aug 22 2022 8:30AM,244604,10,0085942655,ISDIN Corporation,Released
6,Aug 22 2022 8:30AM,244609,10,8346897,Eywa Pharma Inc.,Released
7,Aug 22 2022 8:30AM,244609,10,8347008,Eywa Pharma Inc.,Released
8,Aug 22 2022 8:30AM,244609,10,8346987-BO,Eywa Pharma Inc.,Released
9,Aug 22 2022 8:30AM,244609,10,8346992-BO,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,244602,Released,1
22,244604,Released,6
23,244609,Released,15
24,244610,Released,10
25,244611,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244602,NaN,NaN,1.0
244604,NaN,NaN,6.0
244609,NaN,NaN,15.0
244610,NaN,NaN,10.0
244611,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244530,5.0,0.0,2.0
244533,0.0,0.0,16.0
244544,0.0,0.0,5.0
244551,0.0,0.0,1.0
244555,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244530,5,0,2
244533,0,0,16
244544,0,0,5
244551,0,0,1
244555,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244530,5,0,2
1,244533,0,0,16
2,244544,0,0,5
3,244551,0,0,1
4,244555,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244530,5,,2
1,244533,,,16
2,244544,,,5
3,244551,,,1
4,244555,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 22 2022 8:30AM,244604,10,ISDIN Corporation
6,Aug 22 2022 8:30AM,244609,10,Eywa Pharma Inc.
21,Aug 22 2022 8:30AM,244610,10,Eywa Pharma Inc.
31,Aug 22 2022 8:30AM,244611,10,"Citieffe, Inc."
32,Aug 19 2022 3:29PM,244602,19,ACG North America LLC
33,Aug 19 2022 3:04PM,244600,19,ACG North America LLC
34,Aug 19 2022 2:59PM,244598,20,"ACI Healthcare USA, Inc."
55,Aug 19 2022 2:34PM,244594,20,Alumier Labs Inc.
56,Aug 19 2022 2:34PM,244574,20,Alumier Labs Inc.
66,Aug 19 2022 2:33PM,244533,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 22 2022 8:30AM,244604,10,ISDIN Corporation,,,6
1,Aug 22 2022 8:30AM,244609,10,Eywa Pharma Inc.,,,15
2,Aug 22 2022 8:30AM,244610,10,Eywa Pharma Inc.,,,10
3,Aug 22 2022 8:30AM,244611,10,"Citieffe, Inc.",,,1
4,Aug 19 2022 3:29PM,244602,19,ACG North America LLC,,,1
5,Aug 19 2022 3:04PM,244600,19,ACG North America LLC,,,1
6,Aug 19 2022 2:59PM,244598,20,"ACI Healthcare USA, Inc.",,,21
7,Aug 19 2022 2:34PM,244594,20,Alumier Labs Inc.,,,1
8,Aug 19 2022 2:34PM,244574,20,Alumier Labs Inc.,,,10
9,Aug 19 2022 2:33PM,244533,20,Alumier Labs Inc.,,,16


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 22 2022 8:30AM,244604,10,ISDIN Corporation,6,,
1,Aug 22 2022 8:30AM,244609,10,Eywa Pharma Inc.,15,,
2,Aug 22 2022 8:30AM,244610,10,Eywa Pharma Inc.,10,,
3,Aug 22 2022 8:30AM,244611,10,"Citieffe, Inc.",1,,
4,Aug 19 2022 3:29PM,244602,19,ACG North America LLC,1,,
5,Aug 19 2022 3:04PM,244600,19,ACG North America LLC,1,,
6,Aug 19 2022 2:59PM,244598,20,"ACI Healthcare USA, Inc.",21,,
7,Aug 19 2022 2:34PM,244594,20,Alumier Labs Inc.,1,,
8,Aug 19 2022 2:34PM,244574,20,Alumier Labs Inc.,10,,
9,Aug 19 2022 2:33PM,244533,20,Alumier Labs Inc.,16,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 22 2022 8:30AM,244604,10,ISDIN Corporation,6,,
1,Aug 22 2022 8:30AM,244609,10,Eywa Pharma Inc.,15,,
2,Aug 22 2022 8:30AM,244610,10,Eywa Pharma Inc.,10,,
3,Aug 22 2022 8:30AM,244611,10,"Citieffe, Inc.",1,,
4,Aug 19 2022 3:29PM,244602,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 22 2022 8:30AM,244604,10,ISDIN Corporation,6.0,NaN,NaN
1,Aug 22 2022 8:30AM,244609,10,Eywa Pharma Inc.,15.0,NaN,NaN
2,Aug 22 2022 8:30AM,244610,10,Eywa Pharma Inc.,10.0,NaN,NaN
3,Aug 22 2022 8:30AM,244611,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Aug 19 2022 3:29PM,244602,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 22 2022 8:30AM,244604,10,ISDIN Corporation,6.0,0.0,0.0
1,Aug 22 2022 8:30AM,244609,10,Eywa Pharma Inc.,15.0,0.0,0.0
2,Aug 22 2022 8:30AM,244610,10,Eywa Pharma Inc.,10.0,0.0,0.0
3,Aug 22 2022 8:30AM,244611,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Aug 19 2022 3:29PM,244602,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1467519,35.0,0.0,5.0
16,978220,8.0,0.0,0.0
17,244560,0.0,1.0,0.0
19,489202,2.0,0.0,0.0
20,978299,48.0,0.0,0.0
21,1467541,6.0,0.0,0.0
22,489163,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1467519,35.0,0.0,5.0
1,16,978220,8.0,0.0,0.0
2,17,244560,0.0,1.0,0.0
3,19,489202,2.0,0.0,0.0
4,20,978299,48.0,0.0,0.0
5,21,1467541,6.0,0.0,0.0
6,22,489163,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,35.0,0.0,5.0
1,16,8.0,0.0,0.0
2,17,0.0,1.0,0.0
3,19,2.0,0.0,0.0
4,20,48.0,0.0,0.0
5,21,6.0,0.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,35.0
1,16,Released,8.0
2,17,Released,0.0
3,19,Released,2.0
4,20,Released,48.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,17,19,20,21,22
Status,,,,,,,
Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Released,35.0,8.0,0.0,2.0,48.0,6.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,17,19,20,21,22
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Released,35.0,8.0,0.0,2.0,48.0,6.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,17,19,20,21,22
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Released,35.0,8.0,0.0,2.0,48.0,6.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()